# Modelling User Study 2016

This notebook will contain my attempts at fitting certain models to the user study as I see fit. This is still largely exploratory, and depending on how the model works to explain some of the work we would like to do, I will refine the product further when I can. 

I plan for this notebook to contain parametric and non-parametric models. This means I will assume functional form of some models (using methods like Linear Discriminant Analysis, Regression), and non-parametric analysis (Clustering, K-Nearest Neighbors). The motivation for this is to see if there are insights we can draw from the user study analysis. Parametric models are interpretable. Since I assign variables to study, and the model churns out values for the parameters, I can interpret the relationships between certain parameters and gain some knowledge to use moving forward. 

Non-parametric methods let the machines do that for me. I can narrow down the parameters to study, and let the machine classify users based on their characteristics. For example, if I want to know whether there are distinct usage patterns depending on the area of work the user belongs to, a clustering algorithm can determine that. From there, we can associate usage patterns with area of work.

Let's get to it

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#I will import sci-kit learn procedures as I go.

In [ ]:
#Importing the data file.
data_path  = r"/Users/Owner/Documents/Work_transfer/User Study 2016/"

df = pd.read_csv(data_path+"Clean_File.csv", encoding = "cp1252")

I'm gonna do a split-apply-combine method of cleaning the data up a little bit. This just makes like a little easier for me.

In [ ]:
dfhelp = df.filter(regex = "Help")
helpcategories = []

for col in dfhelp.columns:
    helpcategories.append(dfhelp[col].unique())
    
helpcategories = list(helpcategories[1]) # This has everything we need
helpcategories.remove(helpcategories[1])

In [ ]:
helpcatdict = {}
for cat in helpcategories:
    helpcatdict[cat] = cat[0]

In [ ]:
helpcatdict

In [ ]:
dfhelp = dfhelp.replace(helpcatdict)

In [ ]:
dfhelp.apply(lambda x: x.astype('category', ordered = True)) # Help is now categorical
dfhelp['Participant'] = df['Participant']

In [ ]:
dfwork = df[df.columns[4:10]]

In [ ]:
dfwork = dfwork.apply(lambda x: x.fillna(0)) # That was easy.
dfwork['Participant'] = df['Participant']

In [ ]:
dfno = df.filter(regex = "No")

In [ ]:
dfno.drop(dfno.columns[-3:], inplace = True, axis = 1)
dfno.drop('NoArrangement', inplace = True, axis = 1)

In [ ]:
dfno = dfno.fillna(0) # Swag
dfno['Participant'] = df['Participant']

In [ ]:
dfwhy = df.filter(regex = "Why")

In [ ]:
dfwhy.drop('NoUseWhy', axis = 1, inplace = True)
dfwhy.drop('WhyNotEasy', axis = 1, inplace = True)
dfwhy.drop('WhyUseReason', axis = 1, inplace = True)
dfwhy = dfwhy.fillna(0)

In [ ]:
dfwhy #Ill, NOT sick
dfwhy['Participant'] = df['Participant']

In [ ]:
dfeasy = df.filter(regex = "Easy")

In [ ]:
dfeasy.drop('WhyNotEasy', inplace = True, axis = 1)
dfeasy.drop('EasyOther', inplace = True, axis = 1)

In [ ]:
dfeasy = dfeasy.apply(lambda x: np.where(x == "Yes", 1, 0)) # I filled Na with this too.
dfeasy['Participant'] = df['Participant']

In [ ]:
dfother = df[['Participant', 'Department', 'Community', 'Language', 'Gender']]

In [ ]:
dfother['Department'] = dfother['Department'].astype('category')

In [ ]:
dfother['Community'] = dfother['Community'].astype('category')
dfother['Language'] = dfother['Language'].astype('category')
dfother['Gender'] = dfother['Gender'].astype('category')

In [ ]:
dfother.dtypes #This is what we want

In [ ]:
dfmodel = pd.merge(dfother, dfeasy, on = 'Participant').merge(dfwhy, on = 'Participant').merge(dfno, on = 'Participant').merge(dfwork, on = 'Participant').merge(dfhelp, on = 'Participant')

In [ ]:
# Apparently there is an existing bug with categories, that they lose their category ranking upon a merge.

In [ ]:
dfmodel['Department'] = pd.Categorical(dfmodel['Department'], ordered = False)
dfmodel['Community'] = pd.Categorical(dfmodel['Community'], ordered = False)
dfmodel['Language'] = pd.Categorical(dfmodel['Language'], ordered = False)
dfmodel['Gender'] = pd.Categorical(dfmodel['Gender'], ordered = False)
dfmodel['Helpopen'] = pd.Categorical(dfmodel['Helpopen'], ordered = True)
dfmodel['HelpAgile'] = pd.Categorical(dfmodel['HelpAgile'], ordered = True)
dfmodel['HelpCollab'] =pd.Categorical(dfmodel['HelpCollab'], ordered = True)

In [ ]:
for col in dfmodel.columns:
    if dfmodel[col].dtype == 'float64':
        dfmodel[col] = dfmodel[col].astype(int)

Now I have a clean dataset the makes me happy.

The categorization of some of the variables allows for a much cleaner regression sequence. I likely won't use all of these variables, but they're all there if I want to do anything with it.


In [ ]:
np.set_printoptions(threshold = np.inf)

In [ ]:
print (np.array(dfmodel))

In [ ]:
import statsmodels.api as sm

In [ ]:
dfmodel

In [ ]:
dfmodel.columns

In [ ]:
dftrain = dfmodel[['Community', 'Department', 'Language', 'Gender', 'WhyUseConnect', 'WhyUsePlan', 'WhyUseCoCreate',
                   'WhyUseFeedback', 'WhyUseOrgShareInfo', 'WhyUseFindReUseInfo', 'WhyUseOfficialContent', 'WhyUseFindNewPos',
                   'WhyUseCareerDev', 'WhyUseChat']]

dfpredict = dfmodel['EasyUse']

dfarray = np.array(dftrain)

In [ ]:
np.array(dftrain)

In [ ]:
commdummies = pd.get_dummies(dfmodel['Community'])
depdummies = pd.get_dummies(dfmodel['Department'])
langdummies = pd.get_dummies(dfmodel['Language'])
gendummies = pd.get_dummies(dfmodel['Gender'])

joineddf = commdummies.join(depdummies).join(langdummies).join(gendummies)


In [ ]:
joinedmodel = joineddf.join(dfmodel[['WhyUseConnect', 'WhyUsePlan', 'WhyUseCoCreate',
                   'WhyUseFeedback', 'WhyUseOrgShareInfo', 'WhyUseFindReUseInfo', 'WhyUseOfficialContent', 'WhyUseFindNewPos',
                   'WhyUseCareerDev', 'WhyUseChat']]
)

In [ ]:
joinedmodel

I'm going to see if I can feasibly run a nearest neighbour classifier on this data. A strong prediction score would lead to us knowing there are strong patterns present. Unfortunately, the nearest neighbors score is non-parametric, which means I won't really be able to see what the relationship is.

In [ ]:
from sklearn import neighbors
from sklearn.neighbors import KNeighborsClassifier
for n_neighbors in range(1,26):    
    for weights in ['uniform', 'distance']:
        clf = neighbors.KNeighborsClassifier(n_neighbors, weights=weights) # Ahh cool it's changing the classificaiton in a loop!
        clf.fit(joinedmodel, dfmodel['EasyUse'])

In [ ]:
jma = np.array(joinedmodel) #jma = joinedmodelarray
euma = np.array(dfmodel['EasyUse']) #euma = easyusemodelarray



In [ ]:
indices = np.random.permutation(len(euma))

In [ ]:
# We'll use 4000 to train, 1000 to test and see if this works well

In [ ]:
jma_train = jma[indices[:-860]]
euma_train = euma[indices[:-860]]
jma_test = jma[indices[-860:]]
euma_test = euma[indices[-860:]]


In [ ]:


score_dict = {}
for n in np.arange(1, 50):
    
    for weights in ['uniform', 'distance']:
        
        knn = KNeighborsClassifier(n_neighbors = n, weights = weights)
        
        score_dict[n, weights] = knn.fit(jma_train, euma_train).score(jma_test, euma_test)
        




print (" ")




In [ ]:
score_dict

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfdict = {}
for n in np.arange(500,2500, 500):
    rf = RandomForestClassifier(n_estimators = n)
        
    rfdict[n] = rf.fit(jma_train, euma_train).score(jma_test, euma_test)
    
    

In [ ]:
rfdict

In [ ]:
rf.feature_importances_

I've run the Random Forest Classification in both Python and R now, using different specifications. I used the R model for better interpretability, meaning I used fewer parameters, and they were formatted more nicely. I did have a very bad score however (~49%)

The Python model has a much higher predictability, however I have a hard time interpreting what it means. It has many more variables, and I suspect many of them are highly correlated.


There are a couple things that I need to double check before I go on with any of this. I filled in missing parameters with 0's. Effectively, I prescribed meaning where there effectively was none. These two models are effectively very different because of the assumption that I made.

In [ ]:
euma.sum()/len(euma)

Sci-kit learn runs into dimensionality problems when one-hot encoding many categorical variables, due to its inability to honour categories.

https://roamanalytics.com/2016/10/28/are-categorical-variables-getting-lost-in-your-random-forests/ 

While, for the moment, things worked out nicely, in the future there are some strategies to reduce this:
 - R's randomForest package honours categorical variables automatically, so long as the variable doesn't have high cardinality swithcing to the randomForest package may be the least painful way of running the model
 - Try to find highly related categories. This might be through the data (Principal Component Analysis, which I need to brush up on), or through making predetermined presuppositions. This might be useful in getting the information I need.
 - Even in R, having a vast amount of categorical variables can drastically reduce performance.
 

## Final Random Forest Model

In the interest of transparency of my efforts/lessons learned, I'm keeping all my work here. Below is the final work that I've since completed. I originally fit the model in Spyder, and have since imported it into this notebook.

In both the models in R and Python, I've had issues with the high cardinality of regions and communities and departments. When I tried to fit the model to the large amount of variables, I ran into very low scores. The scores in this data do fairly well, which is nice.

While I found many resources identifying the difficulties of high-cardinality variables in scikit learn, (and R too), there hasn't been any resource to suggest a remedy to this situation. I can't code 
I just dropped the variable, as there are too many options for the amount of data we have left anyway, it gives bad responses.


### What is Random Forest?

Random forest comes from the concept of decision trees. Decision trees try to make decisions on the label of a particular observation by splitting the observations according to certain properties, and to continue doing so until a series of "rules" are created. These decision rules work similarly to how a human would think in a 20 questions game: If you need to guess the word "tiger," you might think something like this:


Is the thing a person? No
Is it an animal? Yes
Is it a mammal? Yes
Does it live on land? Yes
Does it live in North America? No
...
..
Then it must be a tiger!


A decision tree follows that sort of logic but with the data. It tries to classify an unknown observation given what it knows about the data.

The issue with decision trees are the high variance, and the low bias in the process. Essentially, the decision tree designs the rules to make its decision around the training data, and can make its rules too complex, overfitting the model. This means that the decision tree is too well tailored to the data it is given. If you give a different training set of the same observations for the machine to construct a tree out of, it will likely construct a different tree. This is known as variance, where the models' results change drastically given a different sample. However since these decision trees can be sophisticated, they are able to construct relationships very accurately. There is not a very high risk in over-simplifying the tree, since computers are really good at capturing the relationships through trees.


A random forest helps the variance issue, while trying to keep the low bias advantage in the decision tree. The random forest takes MANY samples from the sample data, and creates many trees (usually over 500 decision trees). From the sample, the random forest takes a random sample of the decision parameters in the model, and constructs a tree from them, the repeats this process many times. Then an average of all the trees is used to make a decision about a node.


The averaging process greatly reduces the variance of the model, while still keeping the low bias of the decision tree. It is known as one of the best out-of-the-box classifiers around (although to be fair, many make that claim). Additionally, the way that decision trees make their decisions (out of bag sampling) removes the necessity of N-fold cross-validation. This is handy AND convenient!


In [ ]:
df = pd.read_csv(data_path+"Clean_File.csv", encoding = "latin1") #Reloading the file

df.drop(["Unnamed: 0", "Participant"], inplace = True, axis = 1) # These are useless columns for prediction

#Identify columns to drop - either text questions or too-high cardinality
dropcols = ['DepartmentOther', 'StatusOther', 'UsageLengthOther',
            'OtherResponse', 'WhyUseReason', 'WhyNotEasy', 'FeaturesWant',
            'OtherBenefits', 'Tenure', 'EasyOther', 'Department', 'Community', 'Region']
            
df.drop(dropcols, inplace = True, axis = 1) # These columnds are either textual or have too-high cardinality
# I've ran the models with these features in place with R (which allows for categorical variables)
# There are too many departments to parse through, and the others blow up feature space 


for col in df.columns: # In the survey, the 'float64' variables were binary (0 or 1). In the dataframe, it was represented as
    if df[col].dtype == 'float64': # (1 or NA) which is bad news bears.
        df[col] = df[col].fillna(0) # This fills in the zeroes, but doesn't patch up missing variables.
    
df.dropna(inplace = True, how = 'any') # Since filling in the zeroes where necessary, any blank spaces are now dropped.



df['EasyUse'] = [2 if x == 'Yes' else 1 if x == 'No' else 0 for x in df['EasyUse']]
collist = []
for col in df.columns:
    if df[col].dtype == 'object':
        collist.append(col)



np.random.seed(1) # To split into a random training set.
 # Splitting the values into Yes No or Dunno

df1 = df.drop('EasyUse', axis = 1)
df_with_dummies = pd.get_dummies(df1, columns = collist) # Creating the dummy matrix


easyuse = np.array(df['EasyUse'])
rfmodel =np.array(df_with_dummies) # Looking back, this is almost entirely a sparse matrix, and could have been converted as such
 # But as far as I know, this would only improve performance of the computer, which isn't very necessary

indices = np.random.permutation(len(easyuse)) # The (pseudo)randomly generated list of indices to split the set

easyuse_train = easyuse[indices[:-500]] #Almost 2500 observations, so splitting into almost 80:20
rfmodel_train = rfmodel[indices[:-500]]

easyuse_test = easyuse[indices[-500:]]
rfmodel_test = rfmodel[indices[-500:]]



rfdict = {} # Running the random forests with different parameters for the number of trees
 # Note that Random Forest always uses sqrt(p) parameters for 
for n in np.arange(500,2500, 100):
        rf = RandomForestClassifier(n_estimators = n)
        
        rfdict[n] = rf.fit(rfmodel_train, easyuse_train).score(rfmodel_test, easyuse_test)
    
# Tells us what the most important features are

features_dict = {}
for i in range(len(df_with_dummies.columns)):
    features_dict[df_with_dummies.columns[i]] = rf.feature_importances_[i]
#There we go. This is a strong prediction.


features = pd.DataFrame.from_dict(features_dict, orient = 'index')
features.sort(0, ascending = False)

### Variable Importance

The above text is the "Variable Importance" table of the random forest algorithm. According to Elements of Statistical Learning:


"At each split in each tree, the improvement in the split-criterion is the
importance measure attributed to the splitting variable, and is accumulated
over all the trees in the forest separately for each variable."


Essentially, the importance of a variable is based on how much it improves the split-criterion in the tree. If the variable is important in classifying the observation, it will split the data along the tree very well. In the example above, when we tried to guess "Tiger," the variable ("Animal") would be important, since it helps the guesser eliminate a lot of the options that the secret word is not. However something like ("Is this object on Earth?") would be less important in the classification.

We see above in the table that the most important variables are far and away whether individuals find it easy to find information on GCconnex.

I ran a random forest in R as well (since it is much better suited to categorical variables than Sci-kit learn), and the strongest predictor of whether an individual finds GCconnex easy to use is whether it is easy or not to find information. Additionally, onboarding was an important feature in determining whether or not they found GCconnex useful.



# Weaknesses of this classification:

This model isn't perfect, not by a long shot. There are some complications that reduce the effectiveness of the classification:

- Variables are related
    - Random Forests are best when each variable is unrelated to the other, however in these surveys, this is likely not the case.
    - As a result, there is likely higher variance than there should be in using the Random Forest. This means that there is a greater likelihood that the results would change if we were to re-run the survey and re-run the classification.
  
  
- Blown up feature space (likely with irrelevant variables)
    - Using many irrelevant variables reduces the likelihood that a tree would randomly take a relevant variable in the classification. When using variables like region, or community or department, the feature space gets blown up, resulting in the more relevant variables being taken up less frequently, reducing effectiveness.
    
    
- Imbalance of features
    - In the  "EasyUse" parameter, there are very few "Don't Know / Not Sure" observations. This greatly reduces the accuracy of these classifications, as is apparent from the Confusion matrix. This may also be evident in the independent parameters as well.




### R Code
$ R 
library(randomForest)
library(dplyr)
#The dataset has 86 variables, which is quite a bit. Let's see if we can break this down into something better.
#The reason why I'm using R for this is because of how straight forward the random Forest package. That and RStudio is my fave.
df <- read.csv("/Users/Owner/Documents/Work_transfer/User Study 2016/Clean_File.csv", na.strings = c("", "NA"))

dropcols <- c("DepartmentOther", "UsageLengthOther", "WhyUseReason", "WhyNotEasy", "FeaturesWant", "OtherBenefits", "StatusOther")

df <- df[,!names(df) %in% dropcols] # Dropping the above observations.

df <- df[,-c(1,2)] # Dropping unnecessary indices

df[,2:7][is.na(df[,2:7])] <- 0 
df[,20:28][is.na(df[,20:28])] <- 0


df <- subset(df, select = -c(OtherResponse))
df <- subset(df, select = -c(EasyOther))            

df[,30:40][is.na(df[,30:40])] <- 0

#There, that's the worst of it I think.
df <- df[complete.cases(df),]

df <- df[,-1] # Need to drop department because too many possibilities

train <- sample(1:nrow(df) , 1800)

df.test <- df[-train,"EasyUse"] # Test dataset

set.seed(1)



rf.easy <- randomForest(EasyUse~.,
                         data=df, subset = train, importance = TRUE, ntrees = 2000, proximity = TRUE)
rf.easy
varImpPlot(rf.easy) #This might actually be fairly useful. EasyInfo is by far the most important predictor of whether they
 # find GCconnex useful
importance(rf.easy)

#Plotting the error rates.
plot(rf.easy, main = "Random Forest 'Ease of Use' Prediction Error Rates", lty = 1)
legend("right", colnames(rf.easy$err.rate), col = 1:4, cex = 0.8, fill = 1:4)


MDSplot(rf.easy, df$EasyUse)
 

In [ ]:
## Check out the data visualization workbook for some visual exploration of the results

from sklearn import tree


In [ ]:
classifier = tree.DecisionTreeClassifier()
classifier = classifier.fit(rfmodel_train, easyuse_train)

In [ ]:
classifier

In [ ]:
import pydotplus

In [ ]:

dot_data = tree.export_graphviz(classifier, out_file = data_path+"easy_to_use_tree.dot",
                                feature_names = df_with_dummies.columns, filled = True, rounded = True)

graph = pydotplus.graph_from_dot_file(data_path+"easy_to_use_tree.dot")

In [ ]:
from IPython.display import Image

Image(graph.create_png())
